In [1]:
# YOLO V5 Ultralytics model and config format to Darknet format 
# Adriano Santos.

import torch
import struct
import yaml
import os
from configparser import ConfigParser

# Input files
pt_file_name = "yolov5x.pt"
yaml_file_name = "yolov5x.yaml"

# Output files
cfg_file_name = "yolov5x.cfg"
weights_file_name = "yolov5x.weights"


if os.path.exists(cfg_file_name):
    os.remove(cfg_file_name)
    
if os.path.exists(weights_file_name):
    os.remove(weights_file_name)

'1.7.0+cu110'

In [4]:
# Config file conversion (YAML to CFG)
with open(cfg_file_name, 'w') as cfg:
    with open(yaml_file_name,'r') as f:
        f = yaml.load(f,Loader=yaml.FullLoader)
        net = 0
        for l in f:
            if ((l != 'anchors')and(l !='backbone')and(l !='head')):
                if net == 0:
                    cfg.write('\n[net]\n')
                    cfg.write("width={}\n".format(416))
                    cfg.write("height={}\n".format(416))
                    cfg.write("channels={}\n".format(3))
                    net =1
                cfg.write("{}={}\n".format(l,f[l]))
            elif (l == 'anchors'):
                anchors = []
                for v in f[l]:
                    anchors.extend(v)
                cfg.write("anchors={}\n".format(str(anchors)[1:-1]))
            elif((l == 'backbone') or (l =='head')): 
                for vector in f[l]:
                    #net type
                    cfg.write('\n[{}]\n'.format(vector[2]))
                    #from
                    if type(vector[0]) == int:
                        cfg.write("from={}\n".format(str(vector[0])))
                    elif type(vector[0]) == list:
                        cfg.write("from={}\n".format(str(vector[0])[1:-1]))
                    #number
                    if type(vector[1]) == int:
                        cfg.write("number={}\n".format(str(vector[1])))
                    elif type(vector[1]) == list:
                        cfg.write("number={}\n".format(str(vector[1])[1:-1]))
                    #params
                    if type(vector[3]) == int:
                        cfg.write("args={}\n".format(str(vector[3])))
                    elif type(vector[3]) == list:
                        cfg.write("args={}\n".format(str(vector[3])[1:-1]))

with open(cfg_file_name,'r') as f:
    for l in f.readlines():
        print(l)



[net]

width=416

height=416

channels=3

nc=1

depth_multiple=1.33

width_multiple=1.25

train=./romain/dataporctrain.txt

val=./romain/dataporcvalidation.txt

anchors=10, 13, 16, 30, 33, 23, 30, 61, 62, 45, 59, 119, 116, 90, 156, 198, 373, 326



[Focus]

from=-1

number=1

args=64, 3



[Conv]

from=-1

number=1

args=128, 3, 2



[BottleneckCSP]

from=-1

number=3

args=128



[Conv]

from=-1

number=1

args=256, 3, 2



[BottleneckCSP]

from=-1

number=9

args=256



[Conv]

from=-1

number=1

args=512, 3, 2



[BottleneckCSP]

from=-1

number=9

args=512



[Conv]

from=-1

number=1

args=1024, 3, 2



[SPP]

from=-1

number=1

args=1024, [5, 9, 13]



[BottleneckCSP]

from=-1

number=3

args=1024, False



[Conv]

from=-1

number=1

args=512, 1, 1



[nn.Upsample]

from=-1

number=1

args='None', 2, 'nearest'



[Concat]

from=-1, 6

number=1

args=1



[BottleneckCSP]

from=-1

number=3

args=512, False



[Conv]

from=-1

number=1

args=256, 1, 1



[nn.Upsample]

from=-1

n

In [3]:
# Load model
model = torch.load(pt_file_name, map_location="cpu")['model'].float()  # load to FP32
model.eval()

# Model create
with open(weights_file_name,'w') as f:
    for k,v in model.state_dict().items():
        list_data = list(v.reshape(-1).numpy())
        f.write('{}'.format(k))
        for value in list_data:
            f.write(" ")
            f.write(struct.pack('>f',float(value)).hex())
        f.write('\n')
        
# Model informations
print(model)

Model(
  (model): Sequential(
    (0): Focus(
      (conv): Conv(
        (conv): Conv2d(12, 80, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): Hardswish()
      )
    )
    (1): Conv(
      (conv): Conv2d(80, 160, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(160, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): Hardswish()
    )
    (2): BottleneckCSP(
      (cv1): Conv(
        (conv): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): Hardswish()
      )
      (cv2): Conv2d(160, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (cv3): Conv2d(80, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (cv4): Conv(
        (conv): Conv2d(160, 160, kernel_size=(1, 